In [14]:
#Sid Cox spring 2020

#covid-detector data structure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #basic plotting is good enough here
from Bio import SeqIO


#load up the spreadsheet
#df = pd.read_excel("covid-detector/Human\ Transcription\ Factors.xlsx ")

#pull in the fasta
short_sequences = [] # Setup an empty list

seqfile = "human_tf.fasta"
count =1
for record in SeqIO.parse(seqfile, "fasta"):
    #print(record.id)
    count=count+1
    if len(record.seq) < 3000 :
        # Add this record to our list
        short_sequences.append(record)
print(count)

print("Found %i short sequences" % len(short_sequences))

SeqIO.write(short_sequences, "short_seqs.fasta", "fasta")

8625
Found 6776 short sequences


6776

In [ ]:
class Transcript:
    #primary transcripts of human TFs, will contain multiple isoforms and proteins    
    def __init__(self, name):
        self.name = name

class Isoform:
    #an isoform derived from a primary transcript, will generall contain one or zero proteins
    def __init__(self, name):
        self.name = name

class TF:
    #a  sequence of a transcription factor
    def __init__(self, name, dna):
        self.name = name
        self.dna = dna #the dna sequence
        self.aa = "" #the amino acid sequence